In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_other_major_ratio")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   id           5759 non-null   int64              
 1   ticker       5759 non-null   object             
 2   stac_yymm    5759 non-null   object             
 3   payout_rate  5759 non-null   object             
 4   eva          5759 non-null   object             
 5   ebitda       5759 non-null   object             
 6   ev_ebitda    5759 non-null   object             
 7   updated_at   5759 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(1), object(6)
memory usage: 360.1+ KB


,id,ticker,stac_yymm,payout_rate,eva,ebitda,ev_ebitda,updated_at
0,1,005930,202506,0.01,0.00,342368.00,0.00,2025-09-16 13:03:25.845130+00:00
1,2,005930,202503,0.02,0.00,182041.00,0.00,2025-09-16 13:03:25.845130+00:00
2,3,005930,202412,0.00,29209.00,753568.00,4.84,2025-09-16 13:03:25.845130+00:00
3,4,005930,202409,0.01,0.00,573635.00,0.00,2025-09-16 13:03:25.845130+00:00
4,5,005930,202406,0.01,0.00,371226.00,0.00,2025-09-16 13:03:25.845130+00:00
...,...,...,...,...,...,...,...,...
5754,5755,002710,201903,4.55,0.00,54.00,0.00,2025-09-16 13:03:25.845130+00:00
5755,5756,002710,201812,1.02,53.00,224.00,6.12,2025-09-16 13:03:25.845130+00:00
5756,5757,002710,201809,1.67,0.00,143.00,0.00,2025-09-16 13:03:25.845130+00:00
5757,5758,002710,201806,3.45,0.00,89.00,0.00,2025-09-16 13:03:25.845130+00:00


In [3]:
final_cols = column_index[1:]

In [4]:
df[final_cols] = df[final_cols].replace(r'^\s*$', np.nan, regex=True)
df[final_cols] = df[final_cols].replace(['0', 0], np.nan)

# 결측치 개수 계산
missing_value_counts = df[final_cols].isnull().sum()

# 전체 행의 개수 계산
total_rows = len(df)

# 결측치 비율 계산
missing_value_percent = ((missing_value_counts / total_rows) * 100).round(2)


# --- 3. 결과 취합 및 출력 ---
missing_df = pd.DataFrame({
    'missing_count': missing_value_counts,
    'missing_percent': missing_value_percent
})

# 결측치가 많은 순서대로 정렬하여 출력
missing_df = missing_df[missing_df['missing_count'] > 0]
missing_df = missing_df.sort_values(by='missing_count', ascending=False)

missing_df

,missing_count,missing_percent


In [5]:
missing_cols = missing_df[missing_df['missing_percent'] >= 5].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : []
[원본 컬럼 : 7] - [결측 컬럼 : 0] = [후보 컬럼 : 7]


In [6]:
nunique_counts = df[final_cols].nunique()

unique_cols = nunique_counts[nunique_counts == 1].index.tolist()

final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['updated_at']
[원본 컬럼 : 7] - [결측 컬럼 : 0] - [유니크 컬럼 : 1] = [후보 컬럼 : 6]


In [7]:
df[final_cols]

,ticker,stac_yymm,payout_rate,eva,ebitda,ev_ebitda
0,005930,202506,0.01,0.00,342368.00,0.00
1,005930,202503,0.02,0.00,182041.00,0.00
2,005930,202412,0.00,29209.00,753568.00,4.84
3,005930,202409,0.01,0.00,573635.00,0.00
4,005930,202406,0.01,0.00,371226.00,0.00
...,...,...,...,...,...,...
5754,002710,201903,4.55,0.00,54.00,0.00
5755,002710,201812,1.02,53.00,224.00,6.12
5756,002710,201809,1.67,0.00,143.00,0.00
5757,002710,201806,3.45,0.00,89.00,0.00


In [8]:
print(final_cols)

['ticker', 'stac_yymm', 'payout_rate', 'eva', 'ebitda', 'ev_ebitda']


| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | **필수** | 어떤 기업의 재무 지표인지를 특정하는 핵심 식별자입니다. 모든 데이터를 연결하기 위해 반드시 필요합니다. |
| `stac_yymm` | 결산년월 | **필수** | 데이터가 어느 시점의 실적인지를 나타내는 기준점으로, 시계열 분석 및 특정 시점의 뉴스 이벤트와 결합할 때 필수적입니다. |
| `payout_rate` | 배당성향 | **상** | 기업이 벌어들인 당기순이익 중 얼마를 주주에게 배당금으로 지급했는지를 나타내는 비율입니다. 기업의 주주환원 정책을 보여주는 핵심 지표입니다. |
| `eva` | EVA(경제적부가가치) | **중** | 기업이 영업활동을 통해 창출한 순가치의 증가분으로, 실제 기업의 가치가 얼마나 증가했는지를 보여주는 지표입니다. 장기적 관점에서 중요합니다. |
| `ebitda` | EBITDA | **상** | 세전, 이자지급전, 감가상각전 이익으로, 기업이 영업활동을 통해 벌어들이는 현금흐름 창출 능력을 보여주는 핵심 지표입니다. |
| `ev_ebitda` | EV/EBITDA | **상** | 기업의 시장가치(EV)를 EBITDA로 나눈 값으로, 기업이 벌어들이는 현금흐름 대비 주가가 고평가/저평가되었는지를 판단하는 주요 밸류에이션 지표입니다. |

In [9]:
final_cols = [
  'ticker',
  'stac_yymm',
  'payout_rate',
  'eva',
  'ebitda',
  'ev_ebitda'
]

In [10]:
df[final_cols]

,ticker,stac_yymm,payout_rate,eva,ebitda,ev_ebitda
0,005930,202506,0.01,0.00,342368.00,0.00
1,005930,202503,0.02,0.00,182041.00,0.00
2,005930,202412,0.00,29209.00,753568.00,4.84
3,005930,202409,0.01,0.00,573635.00,0.00
4,005930,202406,0.01,0.00,371226.00,0.00
...,...,...,...,...,...,...
5754,002710,201903,4.55,0.00,54.00,0.00
5755,002710,201812,1.02,53.00,224.00,6.12
5756,002710,201809,1.67,0.00,143.00,0.00
5757,002710,201806,3.45,0.00,89.00,0.00


In [11]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m', '%Y%m%d']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [12]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ticker       5759 non-null   category      
 1   stac_yymm    5759 non-null   datetime64[ns]
 2   payout_rate  5759 non-null   object        
 3   eva          5759 non-null   object        
 4   ebitda       5759 non-null   object        
 5   ev_ebitda    5759 non-null   object        
dtypes: category(1), datetime64[ns](1), object(4)
memory usage: 246.0+ KB


| 컬럼명 | 한글컬럼명 | 현재 타입 | 적정 타입 | 이유 |
| :--- | :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | `category` | `category` | 반복적으로 나타나는 종목 코드는 메모리 효율을 위해 `category` 타입으로 관리하는 것이 가장 좋습니다. |
| `stac_yymm` | 결산년월 | `datetime64[ns]` | `datetime64[ns]` | `YYYYMM` 형식의 날짜 데이터는 `pd.to_datetime` 사용 시 `format='%Y%m'`을 지정하여 `datetime`으로 변환해야 합니다. |
| `payout_rate` | 배당성향 | `object` | `float64` | 소수점을 포함하는 비율 데이터이므로, 부동소수점 타입인 `float64`로 변환해야 정확한 연산이 가능합니다. |
| `eva` | EVA(경제적부가가치) | `object` | `float64` | 소수점을 포함하는 금액 데이터이므로 `float64` 타입이 적합합니다. |
| `ebitda` | EBITDA | `object` | `float64` | 소수점을 포함하는 금액 데이터이므로 `float64` 타입이 적합합니다. |
| `ev_ebitda` | EV/EBITDA | `object` | `float64` | 소수점을 포함하는 밸류에이션 지표이므로 `float64` 타입으로 변환해야 합니다. |

In [13]:
cols_to_check = ['payout_rate', 'eva', 'ebitda', 'ev_ebitda']

# 1. 모든 컬럼을 숫자 타입으로 먼저 변환
df[cols_to_check] = df[cols_to_check].apply(pd.to_numeric, errors='coerce')


# 2. 각 컬럼별로 정수 형태 데이터의 비율을 계산하는 함수를 적용
def get_integer_like_ratio(series: pd.Series) -> float:
  """
  Series의 유효한 값(NaN 제외) 중, 정수 형태인 값의 비율을 계산합니다.
  """
  # NaN 값을 제외한 데이터만 추출
  series_without_na = series.dropna()
  
  # 유효한 데이터가 없는 경우, 1.0 (100%) 반환
  if series_without_na.empty:
    return 1.0
    
  # (정수 형태인 값의 개수) / (전체 유효 데이터의 개수)
  integer_like_count = (series_without_na % 1 == 0).sum()
  total_valid_count = len(series_without_na)
  
  return integer_like_count / total_valid_count

# --- 결과 확인 ---
integer_like_ratio_result = df[cols_to_check].apply(get_integer_like_ratio)
print(integer_like_ratio_result.round(2))

payout_rate    0.15
eva            1.00
ebitda         1.00
ev_ebitda      0.80
dtype: float64


In [14]:
dtype_map = {
  'float64': ['payout_rate', 'eva', 'ebitda', 'ev_ebitda']
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")


In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ticker       5759 non-null   category      
 1   stac_yymm    5759 non-null   datetime64[ns]
 2   payout_rate  5759 non-null   float64       
 3   eva          5759 non-null   float64       
 4   ebitda       5759 non-null   float64       
 5   ev_ebitda    5759 non-null   float64       
dtypes: category(1), datetime64[ns](1), float64(4)
memory usage: 246.0 KB
